# Introduction to PyTerrier

_DSAIT4050: Information retrieval lecture, TU Delft_

**Part 4: Evaluation & experiments**

This part focuses on running experiments and evaluating retrieval and ranking models using PyTerrier. We'll learn how to use query relevances (QRels) to compute ranking metrics and perform statistical significance testing.


In [ ]:
pip install python-terrier==0.12.1

In [2]:
import pyterrier as pt

## Queries and query relevance

Let's start by loading an existing dataset. We'll pick one from [the list](https://pyterrier.readthedocs.io/en/latest/datasets.html#available-datasets) that has topics (queries) and QRels available, such as `vaswani`:


In [3]:
dataset = pt.get_dataset("vaswani")
index = dataset.get_index(variant="terrier_stemmed")

ValueError: Could not find index variant terrier_stemmed for dataset vaswani at http://data.terrier.org/indices/vaswani/terrier_stemmed/latest/files. See available variants at http://data.terrier.org/vaswani.dataset.html

Let's first have a look at the queries (topics) in this dataset. Note that this dataset only has one set of queries and QRels (you can see this in the table). If a dataset has multiple sets, you'll need to specify one, for example: `dataset.get_topics("test")`.


In [ ]:
dataset.get_topics()

Along with the set of queries, we can inspect the corresponding relevance data (QRels). These are tuples of (query, document, label). Note that labels can be either binary (`0` or `1`) or graded (for example, `0` to `3`, where a higher number encodes higher relevance). Tuples where the label is `0` (i.e., no relevance) are often omitted from the QRels.


In [ ]:
dataset.get_qrels()

## Experiments

Using the QRels and the output of a ranking model (i.e., a _ranking_ or a _run_), we can compute ranking metrics that evaluate the quality of our results. Let's start with a BM25 model:


In [6]:
bm25 = pt.terrier.Retriever(index, wmodel="BM25")

An _experiment_ in PyTerrier automates the retrieval/ranking and evaluation process. This means that you can get a comprehensive evaluation of models or systems on some test set in a single line of code:


In [ ]:
from pyterrier.measures import RR, nDCG, MAP

pt.Experiment(
    [bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)

Here, we have evaluted our BM25 retriever on the dataset we loaded earlier. We used three of the most common IR metrics, but [PyTerrier supports a very large number of additional metrics](https://pyterrier.readthedocs.io/en/latest/experiments.html#evaluation-measures-objects) through its integration of the [ir-measures](https://github.com/terrierteam/ir_measures) library.

**Important**: Usually, a value of `0` indicates _no relevance_, while a value greater than `0` indicates _relevance_. However, this is not always the case; a popular example is the TREC Deep Learning passage ranking task, where [only values greater than `1` indicate relevance](https://trec.nist.gov/data/deep2020.html). In those cases, the threshold can by adjusted by constructing the metric as, for example, `RR(rel=2)`.

Note that we used the `@` operator to limit the retrieval depth (number of documents per query) for specific metrics. Furthermore, since both queries (topics) and QRels are simply packaged in a `pandas.DataFrame`, you can easily provide your own data, as long as it is in the right format.

### Performance comparisons

Experiments let you easily compare multiple ranking approaches. Let's load another ranker, which uses standard TF-IDF, to see how much BM25 improves over it:


In [8]:
tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")

We can now run an experiment that evaluates both models on the same data. For better readability, we assign custom names to both approaches.

Comparing the approaches in the same experiment allows us to automatically have _statistical significance testing_ performed. By setting `baseline=0`, we tell the function to compute the $p$-values with respect to the first approach (TF-IDF). Furthermore, [PyTerrier supports a number of correction methods](https://pyterrier.readthedocs.io/en/latest/experiments.html#significance-testing); here, we apply Bonferroni correction:


In [ ]:
pt.Experiment(
    [tf_idf, bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    names=["TF-IDF", "BM25"],
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
    baseline=0,
    correction="bonferroni",
)

### Saving results

Experiments allow for saving the obtained results (runs) in the common TREC format:


In [ ]:
from pathlib import Path

results_dir = Path.cwd() / "04_data" / "results"
results_dir.mkdir(exist_ok=True, parents=True)

pt.Experiment(
    [tf_idf, bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    names=["TF-IDF", "BM25"],
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
    save_dir=str(results_dir),
)

This is useful when results need to be shared or processed futher. In addition, PyTerrier will re-use saved results rather than re-computing them. This behavior can be disabled by setting `save_mode="overwrite"`.

## Further reading

Check out the [section on experiments](https://pyterrier.readthedocs.io/en/latest/experiments.html) in the documentation.
